In [1]:
import pandas as pd
import numpy as np
from dateutil import relativedelta
pd.options.display.max_rows = 100
from datetime import datetime
import re
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import scipy
import json

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import metrics

In [3]:
## local imports

from os import pardir, path
import sys
mod_path = path.abspath(path.join(pardir))

print(mod_path)

if mod_path not in sys.path:
    sys.path.append(mod_path)

print(sys.path)
from data_extract import client, fin, company , companyfinancials_list, pnl, bs, cf, profile, prices, fx, mrktcap, prices_n_fin,  MONGO_FIN ## note needed for this notebook ignore if not using logal Mongodb
from mypackage import Feature, Model


/Users/karimkhalil/Coding/development/stock_portfolio
['/Users/karimkhalil/Coding/development/stock_portfolio/notebooks', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python39.zip', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development/stock_portfolio']
['admin', 'config', 'db_fin', 'local']
['cf', 'prices_bckup', 'company_today', 'mrktcap', 'company_wfinancials', 'prices_n_fin', 'bs', 'fx', 'profile', 'idx_price', 'prices', 'idx_all', 'pnl']
mypackage imported


In [4]:
db = MONGO_FIN()
df_csv = pd.read_csv('../data/data_vf.csv')
# df_csv = pd.read_csv('../data/data_vf_aapl.csv')
df_aapl = df_csv[(df_csv['symbol']=='AAPL') | (df_csv['symbol']=='MSFT')]
feature = Feature(df_aapl)
# feature = Feature(pd.read_csv('../data/data_vf.csv'))
df_raw = feature.df.copy()

Feature object instantiated


In [5]:
feature.check_companyperiod(df_aapl,'symbol', 'year', 'cal_yearperiod', True)

,symbol,year,cal_yearperiod
0,AAPL,1989,1
5,AAPL,1994,3
10,AAPL,1999,3
17,AAPL,2006,3
28,AAPL,2017,3
33,AAPL,2022,1
34,MSFT,1989,2
67,MSFT,2022,1


In [6]:
print(df_csv.shape, df_aapl.shape)

(608641, 128) (257, 128)


In [7]:
# df_aapl = df_csv[df_csv['symbol']=='AAPL']
# df_aapl.to_csv('../data/data_vf_aapl.csv')

In [8]:
# df_returns_test = feature.change(df_csv, 'symbol' , 'cal_yearperiod' , 1, ['usd_adjClose'])
# df_returns_test = feature.change(df_returns_test, 'symbol' , 'cal_yearperiod' , 2, ['usd_adjClose'])
# df_returns_test = df_returns_test[['symbol', 'cal_yearperiod','usd_adjClose' ,  'usd_adjClose_t-1', 'usd_adjClose_t-1_delta' ,'usd_adjClose_t-1_change']]

In [9]:
df_raw.shape

(257, 128)

In [10]:
def replacecol(df: pd.DataFrame, col_replace: str,replace=False):
    df2 = df.copy()
    cols = json.load(open("../config/config.json"))['cols']
    by = cols[col_replace]
    cols = df.columns.tolist()
    col_new = col_replace+"_new"
    if col_new in cols:
        df2.drop(columns=col_new,inplace=True)

    index_col = df2.columns.get_loc(col_replace)
    
    l = []
    l.append(col_replace)
    l.append(col_new)
    before = df2.loc[df2[col_replace]==0].shape[0]
    print(f'{before:,} with zeros for column {col_replace} BEFORE correction')
    if replace==False:
        # df2.insert(index_col, col_new, df2.apply(lambda row: sum([row[i] for i in by]), axis=1)) # overwrite the original data
        ## do not overwrite original data
        df2['calc'] =  df2.apply(lambda row: sum([row[i] for i in by]), axis=1)
        df2.loc[df2[col_replace]==0, col_new] = df2['calc']
        df2.loc[df2[col_replace]!=0, col_new] = df2[col_replace]
    else:
        df2['calc'] = df2.apply(lambda row: sum([row[i] for i in by]), axis=1)
        # df2[col_replace] = df2['calc']
        df2.loc[df2[col_replace]==0, col_new] = df2['calc']
        df2.loc[df2[col_replace]!=0, col_new] = df2[col_replace]
        df2[col_replace] = df2[col_new]
    
    df2['check'] = df2[col_new]- df2[col_replace]
    all_cols = l+by
    after = df2.loc[df2[col_replace]==0].shape[0]
    print(f'{after:,} with zeros for column {col_replace} AFTER correction')
    print("-----------------------------------------")
    print(f'{before-after:,} corrected for column {col_replace}\n')
    return df2[cols], df2[all_cols]

In [11]:
# df_appl_raw = df_raw.loc[df_raw.symbol=='AAPL']
# df_appl_raw.shape

# df_appl_raw = df_raw.loc[df_raw.symbol=='AAPL']
# df_aapl_new = replacecol(df_appl_raw, 'usd_bs_cashAndShortTermInvestments' )[1]
# df_aapl_new = replacecol(df_appl_raw, 'usd_bs_totalCurrentAssets')[1]
# df_aapl_new = replacecol(df_appl_raw, 'usd_bs_goodwillAndIntangibleAssets')[1]
# df_aapl_new = replacecol(df_appl_raw, 'usd_bs_totalNonCurrentAssets', replace=True)[0]

In [12]:
# df_zeroasset = df_csv.loc[(df_csv['usd_bs_totalAssets']==0) & df_csv['usd_bs_cashAndCashEquivalents']!=0]
# print(df_zeroasset.shape)

# df_zeroasset2 = replacecol(df_zeroasset, 'usd_bs_totalAssets', replace=True)[0]
# print(df_zeroasset2.shape[0])

In [13]:
## correct for missisng data

### correct for BS
cols_replace = [
    "usd_bs_cashAndShortTermInvestments",
    "usd_bs_totalCurrentAssets",
    "usd_bs_totalNonCurrentAssets",
    "usd_bs_totalAssets",
    "usd_bs_totalDebt",
    "usd_bs_totalNonCurrentLiabilities",
    "usd_bs_totalCurrentLiabilities",
    "usd_bs_totalLiabilities",
    "usd_bs_totalStockholdersEquity",
    "usd_pnl_sellingGeneralAndAdministrativeExpenses",
    "usd_pnl_operatingExpenses",
    "usd_pnl_costAndExpenses"
    ]

for i in cols_replace:
    df_main = replacecol(feature.df, i, replace=True)[0]

### correct for ebitda
df_main.loc[df_main['usd_pnl_ebitda']==0, 'usd_pnl_ebitda'] = df_main['usd_pnl_revenue'] - df_main['usd_pnl_costAndExpenses']


0 with zeros for column usd_bs_cashAndShortTermInvestments BEFORE correction
0 with zeros for column usd_bs_cashAndShortTermInvestments AFTER correction
-----------------------------------------
0 corrected for column usd_bs_cashAndShortTermInvestments

0 with zeros for column usd_bs_totalCurrentAssets BEFORE correction
0 with zeros for column usd_bs_totalCurrentAssets AFTER correction
-----------------------------------------
0 corrected for column usd_bs_totalCurrentAssets

0 with zeros for column usd_bs_totalNonCurrentAssets BEFORE correction
0 with zeros for column usd_bs_totalNonCurrentAssets AFTER correction
-----------------------------------------
0 corrected for column usd_bs_totalNonCurrentAssets

0 with zeros for column usd_bs_totalAssets BEFORE correction
0 with zeros for column usd_bs_totalAssets AFTER correction
-----------------------------------------
0 corrected for column usd_bs_totalAssets

100 with zeros for column usd_bs_totalDebt BEFORE correction
100 with zeros f

In [14]:
## add returns up to 4 quarters in arrears

df_returns = feature.change(df_main, 'symbol' , 'cal_yearperiod' , 1, ['usd_adjClose'])
df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 2, ['usd_adjClose'])
# df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 3, ['usd_adjClose'])
# df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 4, ['usd_adjClose'])

# df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 1, ['usd_adjClose'], log=True)
# df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 2, ['usd_adjClose'], log=True)
# df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 3, ['usd_adjClose'], log=True)
# df_returns = feature.change(df_returns, 'symbol' , 'cal_yearperiod' , 4, ['usd_adjClose'], log=True)


In [15]:
feature.check_companyperiod(df_returns, 'symbol', 'year', 'cal_yearperiod', True)

,symbol,year,cal_yearperiod
0,AAPL,1989,1
5,AAPL,1994,3
10,AAPL,1999,3
17,AAPL,2006,3
28,AAPL,2017,3
33,AAPL,2022,1
34,MSFT,1989,2
67,MSFT,2022,1


In [16]:
df_returns.shape

(257, 135)

In [17]:
## add global and industry comparatives for 4 quarters returns

### simple returns averages
df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-1_change'])
df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-2_change'])

# df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-3_change'])
# df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-4_change'])

### log returns averages
# df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-1_logchange'])
# df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-2_logchange'])
# df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-3_logchange'])
# df_returns = feature.add_avg(df_returns, ['usd_adjClose_t-4_logchange'])

New DataFrame assigned
New DataFrame assigned


In [18]:
feature.check_companyperiod(df_returns, 'symbol', 'year', 'cal_yearperiod', False)

,symbol,year,cal_yearperiod
0,AAPL,1989,1
1,AAPL,1990,4
2,AAPL,1991,4
3,AAPL,1992,4
4,AAPL,1993,4
5,AAPL,1994,3
6,AAPL,1995,4
7,AAPL,1996,4
8,AAPL,1997,4
9,AAPL,1998,4


In [19]:
## check columns

df_returns.columns.tolist()

['key_shift',
 'Unnamed: 0',
 'Unnamed: 0.1',
 'key',
 'key_main',
 'symbol',
 'companyName',
 'currency',
 'fx_quarter',
 'fx_stock',
 'key_fx',
 'month',
 'year',
 'yearmonth',
 'datetime',
 'date',
 'fin_period',
 'fin_calendarYear',
 'fin_acceptedDateDate',
 'fin_fillingDate',
 'fin_reportedCurrency',
 'cal_period',
 'cal_yearperiod',
 'exchange',
 'exchangeShortName',
 'industry',
 'industry-category',
 'sector',
 'country',
 'usd_adjClose',
 'usd_marketCap',
 'usd_bs_cashAndCashEquivalents',
 'usd_bs_shortTermInvestments',
 'usd_bs_cashAndShortTermInvestments',
 'usd_bs_netReceivables',
 'usd_bs_inventory',
 'usd_bs_otherCurrentAssets',
 'usd_bs_totalCurrentAssets',
 'usd_bs_propertyPlantEquipmentNet',
 'usd_bs_goodwill',
 'usd_bs_intangibleAssets',
 'usd_bs_goodwillAndIntangibleAssets',
 'usd_bs_longTermInvestments',
 'usd_bs_taxAssets',
 'usd_bs_otherNonCurrentAssets',
 'usd_bs_totalNonCurrentAssets',
 'usd_bs_otherAssets',
 'usd_bs_totalAssets',
 'usd_bs_accountPayables',
 'us

In [20]:
df_returns.shape

(257, 144)

In [21]:
print(df_returns.columns.tolist())

['key_shift', 'Unnamed: 0', 'Unnamed: 0.1', 'key', 'key_main', 'symbol', 'companyName', 'currency', 'fx_quarter', 'fx_stock', 'key_fx', 'month', 'year', 'yearmonth', 'datetime', 'date', 'fin_period', 'fin_calendarYear', 'fin_acceptedDateDate', 'fin_fillingDate', 'fin_reportedCurrency', 'cal_period', 'cal_yearperiod', 'exchange', 'exchangeShortName', 'industry', 'industry-category', 'sector', 'country', 'usd_adjClose', 'usd_marketCap', 'usd_bs_cashAndCashEquivalents', 'usd_bs_shortTermInvestments', 'usd_bs_cashAndShortTermInvestments', 'usd_bs_netReceivables', 'usd_bs_inventory', 'usd_bs_otherCurrentAssets', 'usd_bs_totalCurrentAssets', 'usd_bs_propertyPlantEquipmentNet', 'usd_bs_goodwill', 'usd_bs_intangibleAssets', 'usd_bs_goodwillAndIntangibleAssets', 'usd_bs_longTermInvestments', 'usd_bs_taxAssets', 'usd_bs_otherNonCurrentAssets', 'usd_bs_totalNonCurrentAssets', 'usd_bs_otherAssets', 'usd_bs_totalAssets', 'usd_bs_accountPayables', 'usd_bs_shortTermDebt', 'usd_bs_taxPayables', 'usd_b

In [22]:
# construct financial ratios as predictors

df_ratios = feature.add_ratio(df_returns,
    ## profitability ratios
    # GP=('usd_pnl_grossProfit', 'usd_pnl_revenue'),
    EBITDA=('usd_pnl_ebitda' , 'usd_pnl_revenue'),
    # OI=('usd_pnl_operatingIncome' , 'usd_pnl_revenue'),
    # NI = ('usd_pnl_incomeBeforeTax' , 'usd_pnl_revenue'),
    # ROA_OI = ('usd_pnl_operatingIncome' , 'usd_bs_totalAssets'),
    ROA_EBITDA = ('usd_pnl_ebitda' , 'usd_bs_totalAssets'),
    # ROA_NI = ('usd_pnl_incomeBeforeTax' , 'usd_bs_totalAssets'),
    # ROE_OI = ('usd_pnl_operatingIncome' , 'usd_bs_totalStockholdersEquity'),
    # ROE_EBITDA = ('usd_pnl_ebitda' , 'usd_bs_totalStockholdersEquity'),
    # ROE_NI = ('usd_pnl_incomeBeforeTax' , 'usd_bs_totalStockholdersEquity'),
    ## solvency 
    # DE=('usd_bs_totalDebt' , 'usd_bs_totalStockholdersEquity' ),
    DA = ('usd_bs_totalDebt', 'usd_bs_totalAssets'),
    CURR=('usd_bs_totalCurrentAssets' , 'usd_bs_totalCurrentLiabilities'),
    # INTCOV = ('usd_pnl_operatingIncome' , 'usd_pnl_interestExpense'),
    CASH2ASSETS = ('usd_bs_cashAndCashEquivalents' , 'usd_bs_totalAssets'),
    CF2CL=('usd_cf_netCashProvidedByOperatingActivites' , 'usd_bs_totalCurrentLiabilities'),
    CF2D=('usd_cf_netCashProvidedByOperatingActivites' , 'usd_bs_totalDebt'),
    CF2LIAB=('usd_cf_netCashProvidedByOperatingActivites' , 'usd_bs_totalLiabilities'),
    )

df_ratios.shape

New DataFrame assigned
EBITDA
EBITDA_ratio: usd_pnl_ebitda , usd_pnl_revenue
removed 0 rows after EBITDA_ratio
total removed: 0 rows, 0 %

ROA_EBITDA
ROA_EBITDA_ratio: usd_pnl_ebitda , usd_bs_totalAssets
removed 0 rows after ROA_EBITDA_ratio
total removed: 0 rows, 0 %

DA
DA_ratio: usd_bs_totalDebt , usd_bs_totalAssets
removed 0 rows after DA_ratio
total removed: 0 rows, 0 %

CURR
CURR_ratio: usd_bs_totalCurrentAssets , usd_bs_totalCurrentLiabilities
removed 0 rows after CURR_ratio
total removed: 0 rows, 0 %

CASH2ASSETS
CASH2ASSETS_ratio: usd_bs_cashAndCashEquivalents , usd_bs_totalAssets
removed 0 rows after CASH2ASSETS_ratio
total removed: 0 rows, 0 %

CF2CL
CF2CL_ratio: usd_cf_netCashProvidedByOperatingActivites , usd_bs_totalCurrentLiabilities
removed 0 rows after CF2CL_ratio
total removed: 0 rows, 0 %

CF2D
CF2D_ratio: usd_cf_netCashProvidedByOperatingActivites , usd_bs_totalDebt
removed 0 rows after CF2D_ratio
total removed: 0 rows, 0 %

CF2LIAB
CF2LIAB_ratio: usd_cf_netCashProv

(257, 184)

In [23]:
cols_ratios = set(df_ratios.columns.unique())
# cols_returns = set(df_returns.columns.unique())

ratios = [i for i in cols_ratios if i[-5:] == "ratio"]
## list of all ratios created
print(ratios)

# calculate pct change of ratios vs prior period
df_ratios_change = feature.change(df_ratios, 'symbol' , 'cal_yearperiod', 1 , ratios)
print(df_ratios_change.shape)

['pnl_ebitdaratio', 'CF2CL_ratio', 'CF2LIAB_ratio', 'ROA_EBITDA_ratio', 'EBITDA_ratio', 'CASH2ASSETS_ratio', 'CURR_ratio', 'DA_ratio', 'CF2D_ratio']
Error inserting DA_ratio_t-1_change
float division by zero

(257, 210)


In [24]:
change = list(set(df_ratios_change.columns.unique()))

cols_change = [i for i in change if i[-6:] == "change" and i[-8:]!= 'exchange' and i[:12]!= 'usd_adjClose']
print(cols_change)

['EBITDA_ratio_t-1_change', 'ROA_EBITDA_ratio_t-1_change', 'CF2LIAB_ratio_t-1_change', 'CASH2ASSETS_ratio_t-1_change', 'CURR_ratio_t-1_change', 'CF2D_ratio_t-1_change', 'CF2CL_ratio_t-1_change', 'pnl_ebitdaratio_t-1_change']


In [25]:
df_final_v1 = feature.add_avg(df_ratios_change, cols_change)
print(df_final_v1.shape)

New DataFrame assigned
(257, 242)


In [26]:
cols_drop = feature.cols('fin') + feature.cols('ratios') ## columns to be droped keeping featues only
df_final_v2 = df_final_v1.drop(columns=cols_drop)
df_final_v3 = df_final_v2.dropna()


In [27]:
df_final_v2.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257 entries, 0 to 256
Data columns (total 145 columns):
 #    Column                                         Non-Null Count  Dtype  
---   ------                                         --------------  -----  
 0    key_shift                                      257 non-null    object 
 1    Unnamed: 0                                     257 non-null    int64  
 2    Unnamed: 0.1                                   257 non-null    int64  
 3    key                                            257 non-null    object 
 4    key_main                                       257 non-null    object 
 5    symbol                                         257 non-null    object 
 6    companyName                                    257 non-null    object 
 7    currency                                       257 non-null    object 
 8    fx_quarter                                     257 non-null    float64
 9    fx_stock                                 

In [28]:
print(df_final_v2.shape[0], df_final_v3.shape[0])

257 253


In [29]:
df_final_vf = df_final_v2

In [30]:
df_final_periods = df_final_v1.groupby(['year' , 'cal_period'])['symbol'].count().reset_index()
df_final_periods = df_final_periods.groupby('year').agg(NoQ=('cal_period', 'count'), NoCompanies=('symbol', 'sum'))
df_final_periods

,NoQ,NoCompanies
year,,
1989,2,3
1990,4,8
1991,4,8
1992,4,8
1993,4,8
1994,4,7
1995,4,8
1996,4,8
1997,4,8


In [31]:
df_final_v1[df_final_v1.symbol=='AAPL'].groupby('year')['cal_yearperiod'].count()


year
1989    1
1990    4
1991    4
1992    4
1993    4
1994    3
1995    4
1996    4
1997    4
1998    4
1999    3
2000    4
2001    4
2002    4
2003    4
2004    4
2005    4
2006    3
2007    4
2008    4
2009    4
2010    4
2011    4
2012    4
2013    4
2014    4
2015    4
2016    4
2017    3
2018    4
2019    4
2020    4
2021    4
2022    1
Name: cal_yearperiod, dtype: int64

In [42]:
df_aapl_vf = df_final_v2[df_final_v2.symbol=='MSFT']

cols_da = [i for i in df_aapl_vf.columns.tolist() if i[:2] == 'DA']
df_aapl_vf_da = df_aapl_vf[Feature.COLS_BASE+cols_da]

feature.check_companyperiod(df_aapl_vf, 'symbol', 'year', 'cal_yearperiod', False)

,symbol,year,cal_yearperiod
0,MSFT,1989,2
1,MSFT,1990,4
2,MSFT,1991,4
3,MSFT,1992,4
4,MSFT,1993,4
5,MSFT,1994,4
6,MSFT,1995,4
7,MSFT,1996,4
8,MSFT,1997,4
9,MSFT,1998,4


In [41]:
df_aapl_vf = df_final_v3[df_final_v3.symbol=='MSFT']

cols_da = [i for i in df_aapl_vf.columns.tolist() if i[:2] == 'DA']
df_aapl_vf_da = df_aapl_vf[Feature.COLS_BASE+cols_da]

feature.check_companyperiod(df_aapl_vf, 'symbol', 'year', 'cal_yearperiod', False)

,symbol,year,cal_yearperiod
0,MSFT,1990,4
1,MSFT,1991,4
2,MSFT,1992,4
3,MSFT,1993,4
4,MSFT,1994,4
5,MSFT,1995,4
6,MSFT,1996,4
7,MSFT,1997,4
8,MSFT,1998,4
9,MSFT,1999,4


In [ ]:
# df_final_vf = pd.read_csv('../data/df_final_vf2.csv')
# df_final_vf.to_csv('../data/df_final_vwzeros.csv')

In [ ]:
len(df_final_vf.symbol.unique())

In [ ]:
df_final_vf.currency